In [ ]:
%load_ext autoreload
%autoreload 2
import nima_io
import nima_io.read as ir

#import skimage.io
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline
lif = "../tests/data/2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = "../tests/data/t4_1.tif"
img = "../tests/data/exp2_2.tif"

mcts = "../tests/data/multi-channel-time-series.ome.tif"
nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/5.9.1/developers/file-reader.html



In [ ]:
ir.ensure_vm()
md, wr = ir.read(lif)

In [ ]:
wr.rdr.setSeries(4)
wr.rdr.getImageCount()

In [ ]:
md

In [ ]:
rdr, xml_md = ir.img_reader(lif)

In [ ]:
rdr.setSeries(4)
rdr.getImageCount()

In [ ]:
xml_md.getPlaneCount(4), xml_md.getPlaneTheC(4, 110), xml_md.getPixelsSizeZ(4)

# Jpype

In [ ]:
from six.moves.urllib.request import urlopen
import hashlib, os

url = (
    "http://downloads.openmicroscopy.org/bio-formats/"
    + "5.9.0"
    + "/artifacts/loci_tools.jar"
)
loc = "."
path = os.path.join(loc, "loci_tools.jar")
loci_tools = urlopen(url).read()
sha1_checksum = urlopen(url + ".sha1").read().split(b" ")[0].decode()

downloaded = hashlib.sha1(loci_tools).hexdigest()
if downloaded != sha1_checksum:
    raise IOError(
        "Downloaded loci_tools.jar has invalid checksum. " "Please try again."
    )

with open(path, "wb") as output:
    output.write(loci_tools)

In [ ]:
import jpype

# loci_path = '/home/dan/.venvs/imgread-dev/lib/python3.6/site-packages/pims/loci_tools.jar'
# loci_path = '/home/dan/.progs/bftools/bioformats_package.jar'
loci_path = "./loci_tools.jar"
java_memory = "512m"

jpype.startJVM(
    jpype.getDefaultJVMPath(),
    "-ea",
    "-Djava.class.path=" + loci_path,
    "-Xmx" + java_memory,
)

In [ ]:
filepath = img_tile
loci = jpype.JPackage("loci")
rdr = loci.formats.ImageReader()
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(filepath)
xml_md = rdr.getMetadataStore()

In [ ]:
md, mdd = ir.get_md_dict(xml_md, debug=True)
md

In [ ]:
print(md["ImageCount"][0][1], md["PlaneCount"])
print(md["PlanePositionX"][3][1][0])
print(md["ImageAcquisitionDate"])
print(md["PixelsPhysicalSizeX"][0][1][0])
print(md["PixelsType"][0][1])

In [ ]:
md0, rdrdt = ir.read_jpype(img_tile)
rdrdt, md0

In [ ]:
mmd = rdrdt[2]

In [ ]:
mmd["ArcType"]

In [ ]:
rdr = rdrdt[0]

In [ ]:
rdr.getPixelType(), rdr.getDimensionOrder(),

In [ ]:
r = rdr.getMetadataStoreRoot()

In [ ]:
reader = rdr.getReader()

In [ ]:
r.getImage

In [ ]:
md["PlanePositionZ"]

In [ ]:
rdr.getIndex(0, 3, 2)

In [ ]:
loci

In [ ]:
loci = jpype.JPackage("loci")

isLittleEndian = rdr.isLittleEndian()
LE_prefix = [">", "<"][isLittleEndian]
FormatTools = loci.formats.FormatTools
_dtype_dict = {
    FormatTools.INT8: "i1",
    FormatTools.UINT8: "u1",
    FormatTools.INT16: LE_prefix + "i2",
    FormatTools.UINT16: LE_prefix + "u2",
    FormatTools.INT32: LE_prefix + "i4",
    FormatTools.UINT32: LE_prefix + "u4",
    FormatTools.FLOAT: LE_prefix + "f4",
    FormatTools.DOUBLE: LE_prefix + "f8",
}
_dtype_dict_java = {}
for loci_format in _dtype_dict.keys():
    _dtype_dict_java[loci_format] = (
        FormatTools.getBytesPerPixel(loci_format),
        FormatTools.isFloatingPoint(loci_format),
        isLittleEndian,
    )
# above duplicated

In [ ]:
def _jbytearr_javacasting(arr, dtype, bpp, fp, little_endian):
    # let java do the type conversion
    Jconv = loci.common.DataTools.makeDataArray(arr, bpp, fp, little_endian)
    return np.array(Jconv[:], dtype=dtype)


java_dtype = _dtype_dict_java[rdr.getPixelType()]
a_jbytearr_javacasting = lambda arr: _jbytearr_javacasting(arr, dtype, *java_dtype)
a_jbytearr_javacasting([1, 2])

In [ ]:
loci.formats.ImageReader.getPixelType()

In [ ]:
a_jbytearr_javacasting(rdr.openBytes(0)).shape

In [ ]:
%%timeit
rdr.read()

In [ ]:
reader = rdr.reader
# reader=

cs = loci.formats.ChannelSeparator(rdr.reader)
fs = loci.formats.Memoizer(rdr.reader)

In [ ]:
v = meta.getPixelsPhysicalSizeY(0)
v.value(), v.unit, ir.convert_value(v)

In [ ]:
len(readlist), len(md.keys()), len(set(readlist)), len(set(md.keys())), set(
    readlist
) - ss, set(md.keys()) - ss

In [ ]:
md, mdd = ir.read2(
    img_tile,
)

In [ ]:
set(readlist) - set(md), set(md) - set(readlist)

In [ ]:
md.keys(), readlist, md["FolderCount"]

In [ ]:
rdr, xml_md = ir.img_reader(img_tile)

In [ ]:
rdr, xml_md = ir.img_reader(lif)

In [ ]:
rdr.getMetadataStore()

In [ ]:
[(k, v) for k, v in mdd.items() if k[:9] == "getPixels"]

In [ ]:
%%timeit
md, mdd = ir.get_md_dict(xml_md, img_tile)

In [ ]:
class MetadataRetrieve(object):
    """This class is an interface to loci.formats.meta.MetadataRetrieve. At
    initialization, it tests all the MetadataRetrieve functions and it only
    binds the ones that do not raise a java exception.

    Parameters
    ----------
    jmd: jpype._jclass.loci.formats.ome.OMEXMLMetadataImpl
        java MetadataStore, instanciated with:
            jmd = loci.formats.MetadataTools.createOMEXMLMetadata()
        and coupled to reader with `rdr.setMetadataStore(metadata)`

    Methods
    ----------
    <loci.formats.meta.MetadataRetrieve.function>(*args) : float or int or str
        see loci.formats.meta.MetadataRetrieve API on openmicroscopy.org
    """

    def __init__(self, md):
        def wrap_md(fn, name=None, paramcount=None, *args):
            if len(args) != paramcount:
                # raise sensible error for wrong number of arguments
                raise TypeError(
                    ("{0}() takes exactly {1} arguments ({2} " + "given)").format(
                        name, paramcount, len(args)
                    )
                )
            field = fn(*args)

            # deal with fields wrapped in a custom metadata type
            if hasattr(field, "value"):
                field = field.value
            try:  # some fields have to be called
                field = field()
            except TypeError:
                pass

            # check if it is already casted to a python type by jpype
            if not hasattr(field, "toString"):
                return field
            else:
                field = field.toString()

            # convert to int or float if possible
            try:
                return int(field)
            except ValueError:
                try:
                    return float(field)
                except ValueError:
                    return field

        self.fields = []

        for name in dir(md):
            if (name[:3] != "get") or (name in ["getRoot", "getClass"]):
                continue
            fn = getattr(md, name)
            for paramcount in range(5):
                try:
                    field = fn(*((0,) * paramcount))
                    if field is None:
                        continue

                    # If there is no exception, wrap the function and bind.
                    def fnw(fn1=fn, naame=name, paramcount=paramcount):
                        return lambda *args: wrap_md(fn1, naame, paramcount, *args)

                    fnw = fnw()
                    fnw.__doc__ = (
                        "loci.formats.meta.MetadataRetrieve."
                        + name
                        + " wrapped\nby JPype and an "
                        "additional automatic typeconversion.\n\n"
                    )
                    setattr(self, name[3:], fnw)
                    self.fields.append(name[3:])
                    continue
                except:
                    # function is not supported by this specific reader
                    pass

    def __repr__(self):
        return (
            "<MetadataRetrieve> Available loci.formats.meta."
            + "MetadataRetrieve functions: "
            + ", ".join(self.fields)
        )

In [ ]:
mdat.PlaneDeltaT(0, 0)

In [ ]:
mdr = MetadataRetrieve("ChannelAcquisitionMode")
mdr.ChannelAcquisitionMode(0, 0)

In [ ]:
mdr = MetadataRetrieve(meta)
mdr.ChannelAcquisitionMode(0, 0)

In [ ]:
c = meta.getPixelsBinDataBigEndian(0, 10000)
ir.convert_value(c.value)

In [ ]:
ir.get_allvalues_grouped(meta, "getPixelsBigEndian", 1, True)

In [ ]:
md, mdd = ir.get_md_dict(meta)
len(md)

In [ ]:
import javabridge
import bioformats

javabridge.start_vm(class_path=bioformats.JARS)

In [ ]:
%%timeit
im.metadata.PlaneDeltaT(0, 7)

In [ ]:
im = pims.open(img_tile)
im.get_frame(3)

In [ ]:
rdr, xml_md = ir.img_reader("/home/dan/pims/pims/tests/data/bioformats/cluster.nd2")

ht = rdr.getGlobalMetadata()

In [ ]:
md, wr = ir.read2("/home/dan/pims/pims/tests/data/bioformats/cluster.nd2")
md

In [ ]:
from imgread.read import convert_value

In [ ]:
mdata.getPlaneDeltaT(0, 0)

In [ ]:
root = mdata.getRoot()
im0 = root.getImage(0)
pixels = im0.getPixels()

In [ ]:
import io
plane = pixels.getPlane(3)

output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()

In [ ]:
output = io.StringIO()
output.write(str(plane.getDeltaT()))
contents = output.getvalue()
output.close()
contents

In [ ]:
%%timeit

import io

output = io.StringIO()
output.write(str(mdata.getLightPathExcitationFilterRef(0, 0, 0)))
contents = output.getvalue()
output.close()
contents

In [ ]:
md, missing_keys = ir.get_md_dict(mdata, img)

In [ ]:
md["PixelsSizeZ"]

In [ ]:
getattr(mdata, "getChannelExcitationWavelength")(0, 1)

### XML
    getRoot is used in imgread-0.2.1

In [ ]:
ir.ensure_vm()
md, wr = ir.read(img_tile)

In [ ]:
md

In [ ]:
wrapper, mdata = ir.img_reader(img_tile)

In [ ]:
root = mdata.getRoot()

In [ ]:
instrument = root.getInstrument(0)
detector = instrument.getDetector(0)
detector.getModel()

In [ ]:
[m for m in root.methods if m[:3] == "get"]

In [ ]:
image = root.getImage(13)
[m for m in image.methods if m[:3] == "get"]

In [ ]:
obj = image.getObjectiveSettings()
[m for m in obj.methods if m[:3] == "get"]
obj.getID()

In [ ]:
ch = pixels.getChannel(0)
[m for m in ch.methods if m[:3] == "get"]

In [ ]:
lp = ch.getLightPath()
[m for m in lp.methods if m[:3] == "get"]

In [ ]:
ex = lp.getLinkedExcitationFilter(0)
[m for m in ex.methods if m[:3] == "get"]

In [ ]:
ex.getID()

In [ ]:
pixels = image.getPixels()
[m for m in pixels.methods if m[:3] == "get"]

In [ ]:
pixels.getTimeIncrement()

In [ ]:
plane = pixels.getPlane(11)
[m for m in plane.methods if m[:3] == "get"]

In [ ]:
plane.getTheC(), plane.getDeltaT(), plane.getExposureTime()

In [ ]:
for k in root.methods:
    if k[:3] == "get":
        print(k)

In [ ]:
root.getInstrument(0).getDetector(0).getModel()

In [ ]:
mdata.getExperimentID(0)

In [ ]:
root.getExperiment(0)

In [ ]:
im = root.getImage(0)
obj = im.getObjectiveSettings()
obj.getID(), obj.getMedium(), obj.getRefractiveIndex(), obj.getCorrectionCollar(), obj.getObjective()

In [ ]:
mdata.getObjectiveID(0, 0) == obj.getID(), mdata.getObjectiveLensNA(0, 0)

## md vs root (to remember)

from md i can get info on camera, objetives ...

but also key parameters like exposuretime, DeltaT, planePositionXYZ

In [ ]:
metadata=mdata
# might be used to check if metadata are the same for all series
(
    metadata.getObjectiveCount(0),
    metadata.getObjectiveImmersion(0, 0),
    metadata.getPixelsDimensionOrder(0),
    metadata.getPixelsPhysicalSizeX(0),
    # metadata.getTiffDataIFD(0, 5), only fei
    metadata.getObjectiveLensNA(0, 0),
    metadata.getChannelCount(0),
    metadata.getDetectorSettingsBinning(0, 1),  # serie, channel
    metadata.getDetectorModel(0, 0),
    metadata.getDetectorType(0, 0),
    metadata.getTiffDataCount(0),
    metadata.getPlaneExposureTime(0, 1),  # none.value() catch this error with try
    metadata.getPlaneDeltaT(0, 2).value(),
    metadata.getPlaneDeltaT(0, 33).value().doubleValue(),
    metadata.getPlanePositionX(0, 0),
    metadata.getScreenCount(),
    # metadata.getXMLAnnotationCount(),
)
# cerca tutti i ...Count()

In [ ]:
metadata.getXMLAnnotationValue(
    0
) == metadata.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()

In [ ]:
md = bioformats.get_omexml_metadata(lif)
from xml.etree import ElementTree as ETree

mdroot = ETree.fromstring(metadata.dumpXML())
mdroot = ETree.fromstring(
    metadata.getRoot().getStructuredAnnotations().getXMLAnnotation(0).getValue()
)
# mdroot = ETree.fromstring(md)

for a in mdroot:
    print((a.tag, a.attrib))
    for aa in a:
        print((" GGSSS ", aa.tag, aa.attrib))
        for aaa in aa:
            print((" GGSSS2 ", aaa.tag, aaa.attrib))

In [ ]:
import xmltodict

m1 = xmltodict.parse(metadata.dumpXML())
m2 = xmltodict.parse(bioformats.get_omexml_metadata(img_tile))
m2["OME"]

In [ ]:
metalist = [
    (child.tag, [[child.attrib]])
    for child in ETree.fromstring(metadata.dumpXML()).iter()
]

In [ ]:
m = metalist[10]
m[0], m[1][0][0]

In [ ]:
for child in mdroot:
    if child.tag.endswith(""):
        for grandchild in child:
            print((grandchild.tag, grandchild.attrib))